In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
import os.path
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
from pandas.tseries.offsets import MonthEnd
import math
from windrose import WindroseAxes

In [2]:
def calc_intra_annual(df, mean_col, min_col=None, max_col=None, v=False):
    df["DAY"] = df.index.dayofyear
    # next we need to eliminate Feb 29th: 
    df["DAY"] = np.where(df.index.is_leap_year & (df.index.month > 2), df["DAY"] - 1, df["DAY"])  # we just ignore the existence of 29th Feb
    # display(df.loc[(df["DAY"] > 59) & (df["DAY"] <= 61)])
    
    params = ["mean", "std"]
    col_dict = {mean_col: params}
    if min_col is None:
        params.append("min")
    else:
        col_dict[min_col] = "min"
    if max_col is None:
        params.append("max")
    else:
        col_dict[max_col] = "max"
    
    out_df = df.groupby("DAY").agg(col_dict)
    out_df.columns = ("mean", "std", "min", "max")  # "flattens" grouped columns

    out_df["upper"] = out_df["mean"] + out_df["std"]
    out_df["lower"] = out_df["mean"] - out_df["std"]
    
    # checking if the upper and lower values are within max-min limits:
    out_df["upper"] = np.where(out_df["upper"] > out_df["max"], out_df["max"], out_df["upper"])
    out_df["lower"] = np.where(out_df["lower"] < out_df["min"], out_df["min"], out_df["lower"])
    if v:
        display(out_df)
    return out_df


def plot_intra_annual(df, ax):
    ax.fill_between(df.index, df["upper"], df["lower"], color="lightgrey", label="±SD")
    ax.plot(df.index, df["max"], color="grey", linewidth=1, label="max/min")
    ax.plot(df.index, df["min"], color="grey", linewidth=1)
    ax.plot(df.index, df["mean"], color="k", linewidth=1, label="mean")

    
def mark_axes(ax_tuple, px=-0.1, py=1.15):
    letter = "a"
    for ax in ax_tuple:
        ax.text(px, py, "(%s)" % letter, transform=ax.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
        letter = chr(ord(letter) + 1)  # increments the letter in alphabetical order

        
def monthly_ticks(ax, crop=False):
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=16))  # 16 is a slight approximation since months differ in number of days

    ax.xaxis.set_major_formatter(matplotlib.ticker.NullFormatter())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))

    for tick in ax.xaxis.get_minor_ticks():
        tick.tick1line.set_markersize(0)
        tick.tick2line.set_markersize(0)
        tick.label1.set_horizontalalignment("center")
    ax.xaxis.get_minor_ticks()[-1].set_visible(False)
    if crop:
        ax.xaxis.get_minor_ticks()[0].set_visible(False)


In [3]:
cm = 1 / 2.54  # inches to cm
wu = 8.5*cm  # width unit
hu = 6*cm  # height unit

LEAST_SIZE = 7
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

matplotlib.rc('font', size=SMALL_SIZE)          # controls default text sizes
matplotlib.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
matplotlib.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
matplotlib.rc('xtick', labelsize=LEAST_SIZE)    # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
matplotlib.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
matplotlib.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

### Wind roses:

Let's examine the data from the Barentsburg weather station

In [4]:
fig_dir = os.path.join(globals()['_dh'][0], "fig")

in_file = os.path.join(globals()['_dh'][0], "data/wr101248a2.csv")
print(in_file)

dt_parser = lambda x: datetime.strptime(x, "%Y %m %d %H")

tmp = pd.read_csv(in_file, comment="#", parse_dates= {"DATE" : ["YEAR","MONTH","DAY", "HOUR"]}, date_parser=dt_parser)
tmp.set_index("DATE", inplace=True)
tmp.info()

# display(tmp)

/home/tepex/PycharmProjects/Barentsburg-meteo-plots/data/wr101248a2.csv
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 155360 entries, 1965-12-31 21:00:00 to 2020-12-31 18:00:00
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   WMO_ID    155360 non-null  int64  
 1   WIND_DIR  155353 non-null  float64
 2   WS_MEAN   155354 non-null  float64
 3   WS_MAX    123186 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 5.9 MB


In [5]:
tmp["THETA"] = 270 - tmp["WIND_DIR"]
tmp["THETA"] = np.where(tmp["THETA"] < 0, tmp["THETA"] + 360, tmp["THETA"])

tmp["U_WIND"] = tmp["WS_MEAN"] * np.sin(tmp["THETA"] * math.pi / 180)
tmp["U_WIND"] = np.where(tmp["WS_MEAN"] == 0, np.nan, tmp["U_WIND"])

tmp["V_WIND"] = tmp["WS_MEAN"] * np.cos(tmp["THETA"] * math.pi / 180)
tmp["V_WIND"] = np.where(tmp["WS_MEAN"] == 0, np.nan, tmp["V_WIND"])

# display(tmp)

avg_df = tmp.resample("1D").agg({"WS_MEAN": np.mean, "U_WIND": np.mean, "V_WIND": np.mean})
avg_df["WD_DEG"] = np.arctan2(avg_df["V_WIND"], avg_df["U_WIND"]) * 180 / math.pi
avg_df["WD_MEAN"] = np.where(avg_df["WD_DEG"] > 180, avg_df["WD_DEG"] - 180, avg_df["WD_DEG"] + 180)
display(avg_df)

,WS_MEAN,U_WIND,V_WIND,WD_DEG,WD_MEAN
DATE,,,,,
1965-12-31,0.000000,NaN,NaN,NaN,NaN
1966-01-01,0.125000,-6.427876e-01,-0.766044,-130.000000,50.000000
1966-01-02,0.625000,3.061617e-16,-2.500000,-90.000000,90.000000
1966-01-03,0.500000,1.149067e+00,-1.464181,-51.875732,128.124268
1966-01-04,0.750000,-6.840403e-01,-1.879385,-110.000000,70.000000
...,...,...,...,...,...
2020-12-27,3.000000,-2.097280e-01,-2.678818,-94.476627,85.523373
2020-12-28,2.625000,-1.096206e+00,-2.103854,-117.521615,62.478385
2020-12-29,2.875000,8.372405e-02,-2.888377,-88.339659,91.660341


In [ ]:
df = avg_df

df = df.loc[df.index.year >= 2000]

def new_axes():
    fig = plt.figure(figsize=(1.5*wu, 1.5*hu), dpi=300, facecolor='w', edgecolor='w')
    rect = [0.1, 0.1, 0.8, 0.8]
    ax = WindroseAxes(fig, rect)
    fig.add_axes(ax)
    return fig, ax

for y in range(2001, 2021):
    test = df.loc[((df.index.year == y) & (df.index.month.isin((1, 2, 3, 4))) | ((df.index.year == (y - 1)) & (df.index.month.isin((10, 11, 12)))))]

    fig, ax = new_axes()
    ax.bar(test["WD_MEAN"], test["WS_MEAN"], normed=False, opening=0.8, bins=np.arange(0, 25, 5), edgecolor='white')
    ax.set_xticklabels(['E', 'NE', 'N', 'NW',  'W', 'SW', 'S', 'SE'])
    ax.set_yticks(np.arange(10, 60, step=10))
    ax.set_yticklabels(np.arange(10, 60, step=10))
    ax.set_legend()
    ax.set_title("%s-%s" % (y - 1, y))
    
    # fig.tight_layout()
    # fig_path = os.path.join(fig_dir, "windrose_%s-%s.png" % (y - 1, y))
    # plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")